In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import HashingVectorizer


from nltk.tokenize import WordPunctTokenizer
from pymystem3 import Mystem

from tqdm import tqdm

In [2]:
data = pd.read_csv("/kaggle/input/train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [3]:
data.shape

(30000, 4)

In [4]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
def frequency_words(X):
    counter_words = {}
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            for word in X[i][j].split(' '):
                word = word.strip(' ')
                if word in counter_words:
                    counter_words[word] += 1
                else:
                    counter_words[word] = 1
    return counter_words

In [15]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из bow_vocabulary
    указано количество его употреблений
    """ 
    zero_vector = {}
    for i in bow_vocabulary:
        zero_vector[i] = 0
    
    for word in text.split(' '):
        word = word.strip(' ')
        if word in zero_vector:
            zero_vector[word] += 1
    
    return np.array(list(zero_vector.values()))

In [16]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    
    items_bow = []
    for item in range(len(items)):
        item_bow = text_to_bow(items[item][1])
        items_bow.append(item_bow)
    return np.array(items_bow)

In [17]:
def items_to_bow_full(items: np.array) -> np.array:
    items_bow = []
    for item in range(items.shape[0]):
        item_bow_title = text_to_bow(items[item][0])
        item_bow_desc = text_to_bow(items[item][1])
        item_bow = item_bow_title + item_bow_desc
        items_bow.append(item_bow)
    return np.array(items_bow)

### mystem (1.5) балла

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.
Как можно заметить, в текстах одни и те же слова могут быть в разных падежах, а соответственно в bow это будут разные признаки. Чтобы исправить это, можно лемматизировать слова - с помощью библиотеки Mystem.

In [7]:
!pip install pymystem3

In [8]:
mystem = Mystem()

In [9]:
def preprocess_my(text: str) -> str:
    return ' '.join(mystem.lemmatize(text.lower()))

def tokenize_X_my(X):
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            X[i][j] = preprocess_my(X[i][j])
    return X

In [10]:
X_train_my = tokenize_X_my(X_train)
X_test_my = tokenize_X_my(X_test)

In [18]:
bow_vocabulary = frequency_words(X_train_my)
bow_vocabulary = sorted(bow_vocabulary, key=bow_vocabulary.get, reverse=True)[:10000]

In [19]:
X_train_bow_my = items_to_bow_full(X_train_my)
X_test_bow_my = items_to_bow_full(X_test_my)

#### Logistic Regression (при использовании mystem)

In [20]:
bow_model = LogisticRegression().fit(X_train_bow_my, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_my), y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.44


#### SVM (при использовании mystem)

In [21]:
bow_model = LinearSVC().fit(X_train_bow_my, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_my), y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7744444444444445


**Вывод:** качество моделей упало, особенно у LR - с 0.66 до 0.44, а у SVC качество снизилос незначительно с 0.79 до 0.77.

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [22]:
# Посчитаем idf для всех товаров на X_train_my

"""
# Считаем количество того, в скольких документах встретилось каждое из слов из bow_vocabulary
"""

document_freq = {}
for bow_word in bow_vocabulary:
    document_freq[bow_word] = 0


for bow_word in bow_vocabulary:
    for item in X_train_my:
        text = item[0] + ' ' + item[1]
        if bow_word in text:
            document_freq[bow_word] += 1

"""
# Расчет для idf
"""

bow_word_counter = []
idx_bow_words = {}
idx = 0
for bow_word, count in document_freq.items():
    bow_word_counter.append(count)
    idx_bow_words[bow_word] = idx
    idx += 1

idf_vector = np.log(X.shape[0] / np.array(bow_word_counter))

In [23]:
def tfidf(X):

    """
    Расчет для tf
    """
    
    tf_matrix = []
    for item in X:
        tf_item = np.zeros(len(bow_vocabulary))
        text = item[0] + ' ' + item[1]
        for word in text.split(' '):
            if word in bow_vocabulary:
                idx_bow = idx_bow_words[word]
                tf_item[idx_bow] += 1
        tf_item = tf_item / item.shape[0]
        tf_matrix.append(tf_item)

    return np.array(tf_matrix) * idf_vector

In [24]:
X_train_tfidf = tfidf(X_train_my)

In [25]:
X_test_tfidf = tfidf(X_test_my)

In [26]:
# Нормализуйте данные

X_train_tfidf_norm = normalize(X_train_tfidf)
X_test_tfidf_norm = normalize(X_test_tfidf)

### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

#### Logistic Regression (после TF-IDF)

In [27]:
bow_model = LogisticRegression().fit(X_train_tfidf_norm, y_train)
print(accuracy_score(bow_model.predict(X_test_tfidf_norm), y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7635555555555555


#### SVC (после TF-IDF)

In [28]:
bow_model = LinearSVC().fit(X_train_tfidf_norm, y_train)
print(accuracy_score(bow_model.predict(X_test_tfidf_norm), y_test))

0.83


__Вывод__: качество возразло сильно у LR с 0.44 до 0.76, а у SVC с 0.77 до 0.83.

### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [29]:
hv = HashingVectorizer(n_features=10000)

In [30]:
def concat_X(X):
    X_new = []
    for item in X:
        X_new.append(item[0] + ' ' + item[1])
    return np.array(X_new)

In [31]:
X_train_my_conc = concat_X(X_train_my)
X_test_my_conc = concat_X(X_test_my)

In [32]:
X_train_hv = hv.fit_transform(X_train_my_conc)
X_test_hv = hv.fit_transform(X_test_my_conc)

#### Logistic Regression (после Hash)

In [33]:
bow_model = LogisticRegression().fit(X_train_hv, y_train)
print(accuracy_score(bow_model.predict(X_test_hv), y_test))

0.7542222222222222


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### SVC (после Hash)

In [34]:
bow_model = LinearSVC().fit(X_train_hv, y_train)
print(accuracy_score(bow_model.predict(X_test_hv), y_test))

0.8095555555555556


__Вывод__: качество чуть-чуть подупало: у LR с 0.76 до 0.755, а у SVC - с 0.83 до 0.81.

### Word Vectors (1.5 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах изинтернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [35]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-24 17:28:36--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-24 17:28:37--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9c5142e2c0254c4d3dfa5c886b.dl.dropboxusercontent.com/cd/0/inline/A2cH8oDrMetX-LbrBdwDUPbYQIupRqSdik9bNbGdBwIcNUeGLae4lszQoepDD4-hbrD3hJ65tJI5b5vIFpnl8yDEPmcxmZCi2uMKk8fcPMOnXQ/file# [following]
--2020-04-24 17:28:37--  https://uc9c5142e2c0254c4d3dfa5c886b.dl.dropboxusercontent.com/cd/0/inline/A2cH8oDrMetX-LbrBdwDUPbYQIupRqSdik9bNbGdBwIcNUeGLae4lszQoepDD4-hbrD3hJ65tJI5b5vIFpnl8yDEPmcxmZCi2uMKk8fcPMOnXQ/file
Resolving uc9c5142e2c0254c4d

In [36]:
!tar -xzf ru.tar.gz

tar: ru.vec: Wrote only 5632 of 8861 bytes
tar: Exiting with failure status due to previous errors


In [37]:
!ls

__notebook_source__.ipynb  ru.bin  ru.tar.gz  ru.tar.gz.1  ru.tar.gz.2	ru.vec


In [38]:
import gensim
from gensim.models.wrappers import FastText


model = FastText.load_fasttext_format('ru.bin')

In [39]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [40]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [41]:
def embedding(X) -> np.array:
        
    embedding_dim = model['кек'].shape[0]
    
    X_embed = []
    for item in X:
        features = np.zeros([embedding_dim], dtype='float32')
        text = item[0] + ' ' + item[1]
        for word in text.split(' '):
            if word in model:
                features += model[word]
        X_embed.append(features)

    return np.array(X_embed)

In [42]:
X_train_embed = embedding(X_train_my)

In [43]:
X_test_embed = embedding(X_test_my)

#### Logistic Regression (после Word Vectors)

In [44]:
bow_model = LogisticRegression().fit(X_train_embed, y_train)
print(accuracy_score(bow_model.predict(X_test_embed), y_test))

0.5828888888888889


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### SVC (после Word Vectors)

In [45]:
bow_model = LinearSVC().fit(X_train_embed, y_train)
print(accuracy_score(bow_model.predict(X_test_embed), y_test))

0.6145555555555555


/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


__Вывод__: по сравнению с качеством на Hash: качество резко упало у обоих моделей: у LR - с 0.755 до 0.58, а у SVC - 0.81 до 0.61.